## Periodic pattern mining on canadian TV logs
<img src="skmine_series.png" alt="logo" style="width: 60%;"/>

### The problem, informally
Let's take a simple example. 

Imagine you set an alarm to wake up every day around 7:30AM, and go to work. Sometimes you wake up a bit earlier (your body anticipates on the alarm), and sometimes a bit later, for example if you press the "snooze" button and refuse to face the fact that you have to wake up.

In python we can load those "wake up" events as logs, and store them in a [pandas.Series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html), like

In [1]:
import datetime as dt
import pandas as pd
one_day = 60 * 24  # a day in minutes
minutes = [0, one_day - 1, one_day * 2 - 1, one_day * 3, one_day * 4 + 2, one_day * 7]

S = pd.Series("wake up", index=minutes)
start = dt.datetime.strptime("16/04/2020 07:30", "%d/%m/%Y %H:%M")
S.index = S.index.map(lambda e: start + dt.timedelta(minutes=e))
S.index = S.index.round("min")  # minutes as the lowest unit of difference
S

2020-04-16 07:30:00    wake up
2020-04-17 07:29:00    wake up
2020-04-18 07:29:00    wake up
2020-04-19 07:30:00    wake up
2020-04-20 07:32:00    wake up
2020-04-23 07:30:00    wake up
dtype: object

In [2]:
S.index.to_numpy()

array(['2020-04-16T07:30:00.000000000', '2020-04-17T07:29:00.000000000',
       '2020-04-18T07:29:00.000000000', '2020-04-19T07:30:00.000000000',
       '2020-04-20T07:32:00.000000000', '2020-04-23T07:30:00.000000000'],
      dtype='datetime64[ns]')

We can see the wake-up time is not exactly the same every day, but overall a consistent regular pattern seems to emerge.

Now imagine that in addition to wake up times, we also have records of other daily activities (meals, work, household chores, etc.), and that rather than a handful of days, those records span several years and make up several thousands of events

**How would you be able to detect regularities in the data ?**

### Introduction to periodic pattern mining
Periodic pattern mining aims at exploiting regularities not only about `what happens` by finding coordinated event occurrences, but also about `when it happens` and `how it happens`, by **finding consistent inter-occurrence timeintervals**.

Next, we introduce the concept of cycles

#### The cycle : a building block for periodic pattern mining
Here is an explicit example of a cycle

<img src="cycle_color.png" alt="cycle" style="width: 60%;"/>

This definition, while being relatively simple, is general enough to allow us to find regularities in different types of logs

#### Handling noise in our timestamps

Needless to say, it would be too easy if events in our data were equally spaced. As data often comes noisy, we have to be fault tolerant, and allow small errors to sneak into our cycles. 

That's the role of `shift corrections`, which capture the small deviations from perfectly regular periodic repetitions, and allow to reconstruct the (noisy) original sequence of events, using the following relation
<img src="shifts.png" alt="shifts" style="width: 60%;"/>

#### A tiny example with scikit-mine
`scikit-mine` offers a `PeriodicCycleMiner`, out of the box.
You can use it to **detect regularities, in the form of cycles**, in the input data. 

These regularities are submitted to an MDL criterion, so that we do not mistakenly include redundant occurences, nor forget to consider other intervals that would sumarize our data in a better way.

MDL offers a framework to find `the best set of cycles`, i.e the set that gives the most succint representation of the data. And `as humans, we often like to deal with non-redundant, well organized data`.

In [3]:
from skmine.periodic import PeriodicCycleMiner
import pandas as pd
S_doc = pd.Series("ring_a_bell", [10, 20, 32, 40, 60, 79, 100, 240])
pcm = PeriodicCycleMiner().fit(S_doc)
pcm.discover()

self.alpha_groups 1
S.values 8
 ---- COLLECTION STATS (Total=1 nb_simple=1)
Code length patterns (1): 30.826182
Code length residuals (3): 23.555247
-- Total code length = 54.381429 (86.575343% of 62.813992)



/home/cregan/fsl/lib/python3.10/site-packages/scikit_mine-0.0.10-py3.10.egg/skmine/periodic/cycles.py:20: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


,t0,pattern_json_tree,pattern_resume,length_major,period_major,cost,type,E
0,20,"{""nid"": 0, ""r"": 5, ""p"": 20, ""parent"": ""None"", ...",(ring_a_bell)[r=5 p=20],5,20,30.826182,simple,2


In [4]:
from skmine.periodic import PeriodicCycleMiner
import pandas as pd

In [5]:
pcm = PeriodicCycleMiner().fit(S)

self.alpha_groups 1
S.values 6
 ---- COLLECTION STATS (Total=1 nb_simple=1)
Code length patterns (1): 67.885186
Code length residuals (1): 15.884194
-- Total code length = 83.769381 (87.895949% of 95.305166)



In [6]:
pcm.discover()

,t0,pattern_json_tree,pattern_resume,length_major,period_major,cost,type,E
0,2020-04-16 07:30:00,"{""nid"": 0, ""r"": 5, ""p"": 8643, ""parent"": ""None""...",(wake up)[r=5 p=8643],5,1 days 00:00:30,67.885186,simple,24


You can see one cycle has been extracted for our event `wake up`. The cycle covers the entire business week, but not the last monday separated by the weekend

It has a length of 5 and a period close to 1 day, as expected.

Also, note that we "lost" some information here. Our period of 1 day offers the best summary for this data.
Accessing the little "shifts" as encountered in original data is also possible, with an extra argument in our `.discover` call

The last column named `dE` contains a list of shifts to apply to our cycle in case we want to reconstruct the original data. Trailing zeros have been removed for efficiency, and their values are `relative to the period`, but we can see there is:
 * a -90 second shift between the 1st and 2nd entry (1day30s - 90s later = waking up at 7:29 on tuesday)
 * a 30 second shift between the 2nd and 3rd entry (1day30s - 30s later = still waking up at 7:29 on wednesday)
 * an 30 second shift between the 3rd and 4th entry (back to 7:30 on thursday)
 * an 90 second shift between the 4th and 5th entry (1day 30s + 90s later = waking up at 7:32 on friday)

Also note that we can get the "uncovered" events, called `redisuals`

In [7]:
pcm.get_residuals()

,time,event
0,2020-04-23 07:30:00,wake up


This way `pcm` does not store all the data, but has all information needed to reconstruct it entirely !!

In [8]:
pcm.reconstruct()

,time,event
0,2020-04-16 07:30:00,wake up
1,2020-04-17 07:29:00,wake up
2,2020-04-18 07:29:00,wake up
3,2020-04-19 07:30:00,wake up
4,2020-04-20 07:32:00,wake up


### An example with Canadian TV programs
#### Fetching logs from canadian TV

In this section we are going to load some event logs of TV programs (the `WHAT`), indexed by their broadcast timestamps (the `WHEN`).

`PeriodicCycleMiner` is here to help us discovering regularities (the `HOW`)

In [9]:
from skmine.datasets import fetch_canadian_tv
from skmine.periodic import PeriodicCycleMiner

#### Searching for cycles in TV programs

Remember about the definition of cycles ?
Let's apply it to our TV programs

In our case

* $\alpha$ is the name of a TV program

* $r$ is the number of broadcasts (repetitions) for this TV program (inside this cycle)

* $p$ is the optimal time delta between broadcasts in this cycle. If a program is meant to be live everyday at 14:00PM, then $p$ is likely to be `1 day`

* $\tau$ is the first broadcast time in this cycle

* $dE$ are the shift corrections between the $p$ and the actual broadcast time of an event. If a TV program was scheduled at 8:30:00AM and it went on air at 8:30:23AM the same day, then we keep track of a `23 seconds shift`. This way we can summarize our data (via cycles), and reconstruct it (via shift corrections). 


Finally we are going to dig a little deeper into these cycles, to answer quite complex questions about our logs. We will see that cycles contains usefull information about our input data

In [10]:
ctv_logs = fetch_canadian_tv()
ctv_logs.head()

/home/cregan/fsl/lib/python3.10/site-packages/scikit_mine-0.0.10-py3.10.egg/skmine/datasets/periodic.py:113: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  s = pd.read_csv(


timestamp
2020-08-01 06:00:00            The Moblees
2020-08-01 06:11:00    Big Block Sing Song
2020-08-01 06:13:00    Big Block Sing Song
2020-08-01 06:15:00               CBC Kids
2020-08-01 06:15:00               CBC Kids
Name: canadian_tv, dtype: string

Compute only simple cycles on ctv_logs with the karg *complex=False* : 

In [11]:
pcm = PeriodicCycleMiner().fit(ctv_logs, complex=False)
pcm.discover()

/home/cregan/fsl/lib/python3.10/site-packages/scikit_mine-0.0.10-py3.10.egg/skmine/periodic/cycles.py:121: UserWarning: Duplicates found in the input sequence, they have been removed.
  warnings.warn(


self.alpha_groups 98
S.values 1859
 ---- COLLECTION STATS (Total=298 nb_simple=298)
Code length patterns (298): 17740.810901
Code length residuals (479): 11285.019585
-- Total code length = 29025.830486 (67.942879% of 42720.931352)



,t0,pattern_json_tree,pattern_resume,length_major,period_major,cost,type,E
57,2020-08-01 04:00:00,"{""nid"": 0, ""r"": 5, ""p"": 60480, ""parent"": ""None...",(Rick Mercer Report)[r=5 p=60480],5,7 days 00:00:00,54.264637,simple,0
58,2020-08-01 04:30:00,"{""nid"": 0, ""r"": 5, ""p"": 60480, ""parent"": ""None...",(Rick Mercer Report)[r=5 p=60480],5,7 days 00:00:00,54.264637,simple,0
0,2020-08-01 05:00:00,"{""nid"": 0, ""r"": 31, ""p"": 8640, ""parent"": ""None...",(Grand Designs)[r=31 p=8640],31,1 days 00:00:00,101.690178,simple,0
65,2020-08-01 06:00:00,"{""nid"": 0, ""r"": 5, ""p"": 60480, ""parent"": ""None...",(The Moblees)[r=5 p=60480],5,7 days 00:00:00,54.264637,simple,0
25,2020-08-01 06:11:00,"{""nid"": 0, ""r"": 5, ""p"": 60480, ""parent"": ""None...",(Big Block Sing Song)[r=5 p=60480],5,7 days 00:00:00,54.264637,simple,0
...,...,...,...,...,...,...,...,...
277,2020-08-26 04:00:00,"{""nid"": 0, ""r"": 3, ""p"": 7560, ""parent"": ""None""...",(CBC News: The National)[r=3 p=7560],3,0 days 21:00:00,54.585597,simple,0
231,2020-08-26 10:59:00,"{""nid"": 0, ""r"": 4, ""p"": 8640, ""parent"": ""None""...",(CBC Kids)[r=4 p=8640],4,1 days 00:00:00,67.937446,simple,12
276,2020-08-26 14:00:00,"{""nid"": 0, ""r"": 3, ""p"": 8640, ""parent"": ""None""...",(Jamie's Super Foods)[r=3 p=8640],3,1 days 00:00:00,54.573179,simple,0
190,2020-08-27 02:00:00,"{""nid"": 0, ""r"": 4, ""p"": 180, ""parent"": ""None"",...",(Mr. D)[r=4 p=180],4,0 days 00:30:00,56.081774,simple,0


Compute simple and complex (with horizontal and vertical combinations) cycles on ctv_logs : 

In [12]:
pcm = PeriodicCycleMiner().fit(ctv_logs)

/home/cregan/fsl/lib/python3.10/site-packages/scikit_mine-0.0.10-py3.10.egg/skmine/periodic/cycles.py:121: UserWarning: Duplicates found in the input sequence, they have been removed.
  warnings.warn(


self.alpha_groups 98
S.values 1859
 ---- COLLECTION STATS (Total=140 nb_simple=70 nb_concat=60 nb_other=8 nb_nested=2)
Code length patterns (140): 13747.284696
Code length residuals (413): 9800.106879
-- Total code length = 23547.391575 (55.119097% of 42720.931352)



In [13]:
cycles = pcm.discover()
cycles

,t0,pattern_json_tree,pattern_resume,length_major,period_major,cost,type,E
51,2020-08-01 04:00:00,"{""nid"": 0, ""r"": 5, ""p"": 60480, ""parent"": ""None...",(Rick Mercer Report)[r=5 p=60480],5,7 days 00:00:00,54.264637,simple,0
45,2020-08-01 04:30:00,"{""nid"": 0, ""r"": 5, ""p"": 60480, ""parent"": ""None...",(Rick Mercer Report [d=642] The Adventures of ...,5,7 days 00:00:00,103.131763,concat,6
0,2020-08-01 05:00:00,"{""nid"": 0, ""r"": 31, ""p"": 8640, ""parent"": ""None...",(Grand Designs)[r=31 p=8640],31,1 days 00:00:00,101.690178,simple,0
9,2020-08-01 06:00:00,"{""nid"": 0, ""r"": 5, ""p"": 60480, ""parent"": ""None...",(The Moblees [d=66] Big Block Sing Song [d=12]...,5,7 days 00:00:00,164.529690,concat,6
96,2020-08-01 06:30:00,"{""nid"": 0, ""r"": 5, ""p"": 60477, ""parent"": ""None...",(Daniel Tiger's Neighbourhood)[r=5 p=60477],5,6 days 23:59:30,72.264958,simple,18
...,...,...,...,...,...,...,...,...
93,2020-08-25 02:00:00,"{""nid"": 0, ""r"": 4, ""p"": 180, ""parent"": ""None"",...",(This Hour Has 22 Minutes)[r=4 p=180],4,0 days 00:30:00,56.081774,simple,0
24,2020-08-26 00:30:00,"{""nid"": 0, ""r"": 7, ""p"": 180, ""parent"": ""None"",...",(Kim's Convenience)[r=7 p=180],7,0 days 00:30:00,61.078848,simple,0
105,2020-08-26 14:00:00,"{""nid"": 0, ""r"": 3, ""p"": 8640, ""parent"": ""None""...",(Jamie's Super Foods [d=1980] Just For Laughs:...,3,1 days 00:00:00,94.162808,concat,0
91,2020-08-27 02:00:00,"{""nid"": 0, ""r"": 4, ""p"": 180, ""parent"": ""None"",...",(Mr. D)[r=4 p=180],4,0 days 00:30:00,56.081774,simple,0


`Note` : no need to worry for the warning, it's here to notify duplicate event/timestamp pairs have been found

Now that we have our cycles in a [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html), we can play with the pandas API and answer questions about our logs

#### Did I find cycles for the TV show "Arthurt Shorts"

In [14]:
cycles

,t0,pattern_json_tree,pattern_resume,length_major,period_major,cost,type,E
51,2020-08-01 04:00:00,"{""nid"": 0, ""r"": 5, ""p"": 60480, ""parent"": ""None...",(Rick Mercer Report)[r=5 p=60480],5,7 days 00:00:00,54.264637,simple,0
45,2020-08-01 04:30:00,"{""nid"": 0, ""r"": 5, ""p"": 60480, ""parent"": ""None...",(Rick Mercer Report [d=642] The Adventures of ...,5,7 days 00:00:00,103.131763,concat,6
0,2020-08-01 05:00:00,"{""nid"": 0, ""r"": 31, ""p"": 8640, ""parent"": ""None...",(Grand Designs)[r=31 p=8640],31,1 days 00:00:00,101.690178,simple,0
9,2020-08-01 06:00:00,"{""nid"": 0, ""r"": 5, ""p"": 60480, ""parent"": ""None...",(The Moblees [d=66] Big Block Sing Song [d=12]...,5,7 days 00:00:00,164.529690,concat,6
96,2020-08-01 06:30:00,"{""nid"": 0, ""r"": 5, ""p"": 60477, ""parent"": ""None...",(Daniel Tiger's Neighbourhood)[r=5 p=60477],5,6 days 23:59:30,72.264958,simple,18
...,...,...,...,...,...,...,...,...
93,2020-08-25 02:00:00,"{""nid"": 0, ""r"": 4, ""p"": 180, ""parent"": ""None"",...",(This Hour Has 22 Minutes)[r=4 p=180],4,0 days 00:30:00,56.081774,simple,0
24,2020-08-26 00:30:00,"{""nid"": 0, ""r"": 7, ""p"": 180, ""parent"": ""None"",...",(Kim's Convenience)[r=7 p=180],7,0 days 00:30:00,61.078848,simple,0
105,2020-08-26 14:00:00,"{""nid"": 0, ""r"": 3, ""p"": 8640, ""parent"": ""None""...",(Jamie's Super Foods [d=1980] Just For Laughs:...,3,1 days 00:00:00,94.162808,concat,0
91,2020-08-27 02:00:00,"{""nid"": 0, ""r"": 4, ""p"": 180, ""parent"": ""None"",...",(Mr. D)[r=4 p=180],4,0 days 00:30:00,56.081774,simple,0


In [15]:
cycles[cycles["pattern_resume"].apply(lambda x: "Arthur Shorts" in x)]

KeyError: 'event'

#### What are the top 10 longest cycles ?

In [ ]:
cycles.nlargest(10, ["length_major"])

In [ ]:
pcm.reconstruct([1])

In [ ]:
pcm.get_residuals()

#### what are the 10 most unpunctual TV programs ?
For this we are going to :
 1. extract the shift corrections along with other informations about our cycles
 2. compute the sum of the absolute values for the shift corrections, for every cycles
 3. get the 10 biggest sums

In [ ]:
# full_cycles = pcm.discover()
# full_cycles.head()

In [ ]:
# def absolute_sum(*args):
#     return sum(map(abs, *args))

# # level 0 is the name of the TV program
# shift_sums = full_cycles["dE"].map(absolute_sum).groupby(level=[0]).sum()
# shift_sums.nlargest(10)

<!-- #### what are the 10 most unpunctual TV programs ?
For this we are going to :
 1. extract the shift corrections along with other informations about our cycles
 2. compute the sum of the absolute values for the shift corrections, for every cycles
 3. get the 10 biggest sums -->

In [ ]:
# def absolute_sum(*args):
#     return sum(map(abs, *args))

# # level 0 is the name of the TV program
# shift_sums = full_cycles["dE"].map(absolute_sum).groupby(level=[0]).sum()
# shift_sums.nlargest(10)

#### What TV programs have been broadcasted every day for at least 5 days straight?
Let's make use of the [pandas.DataFrame.query](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html) method to express our question in an SQL-like syntax

In [ ]:
# cycles.query('length_major >= 5 and period_major >= 3600', engine='python')
# # cycles.query('length >= 5 and period == "1 days"', engine='python')

#### What TV programs are broadcast only on business days ?
From the previous query we see we have a lot of 5-length cycles, with periods of 1 day.
An intuition is that these cycles take place on business days. Let's confirm this by considering cycles with
 1. start timestamps on mondays
 2. periods of roughly 1 day  

In [ ]:
# monday_starts = cycles[cycles.start.dt.weekday == 0]  # start on monday
# monday_starts.query('length == 5 and period == "1 days"', engine='python')

References
----------

1.
    Galbrun, E & Cellier, P & Tatti, N & Termier, A & Crémilleux, B
    "Mining Periodic Pattern with a MDL Criterion"

2.
    Galbrun, E
    "The Minimum Description Length Principle for Pattern Mining : A survey"

3. 
    Termier, A
    ["Periodic pattern mining"](http://people.irisa.fr/Alexandre.Termier/dmv/DMV_Periodic_patterns.pdf) 